In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import os, os.path
import time
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from urllib.request import Request
import requests
import regex as re
import unicodedata

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# folder path

In [3]:
# URL to scrap
# url = 'https://fbref.com/en/matches/dadb1883/Spain-Costa-Rica-November-23-2022-World-Cup'
url ='https://fbref.com/en/matches/9397f79c/Spain-Germany-November-27-2022-World-Cup'

# headers
headers={'User-Agent': 'Mozilla/5.0'}

# requests
my_url = Request(url, headers=headers)
uClient = uReq(my_url)
time.sleep(0.5)
page_html = uClient.read()
time.sleep(0.5)
uClient.close()

# html parsing
soup = soup(page_html, "html.parser")

In [4]:
tables = soup.find_all('table')
# tables

In [5]:
# page_soup.findAll("div",{'id': 'team_stats'})
for table in soup.find_all('table'):
    print(table.get('class'))

None
None
None
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['min_width', 'sortable', 'stats_table', 'min_width', 'shade_zero']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['min_width', 'sortable', 'stats_table', 'min_width', 'shade_zero']
['stats_table', 'sortable', 'min_width']
['stats_table', 'sortable', 'min_width']
['stats_table', 'sortable', 'min_width']


In [6]:
table = soup.find('table', class_='stats_table sortable')

In [7]:
df = pd.DataFrame(columns=['Player', '#', 'Pos', 'Age', 'Min',
                          'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks',
                          'xG', 'npxG', 'xAG',
                          'SCA','GCA', 'P_Cmp'
                           , 'P_Att', 'P_Cmp%',
                           'D_Succ', 'D_Att'])
df

,Player,#,Pos,Age,Min,Gls,Ast,PK,PKatt,Sh,...,xG,npxG,xAG,SCA,GCA,P_Cmp,P_Att,P_Cmp%,D_Succ,D_Att


In [8]:
team_1 = df[['Player', '#', 'Pos', 'Age', 'Min',
          'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks',
          'P_Att','P_Cmp', 'P_Cmp%',
          'D_Att', 'D_Succ'
            ]]
team_1

,Player,#,Pos,Age,Min,Gls,Ast,PK,PKatt,Sh,...,CrdR,Touches,Tkl,Int,Blocks,P_Att,P_Cmp,P_Cmp%,D_Att,D_Succ


In [9]:
player_name = []
for name in table.tbody.find_all('th'):
    try:
        player_name.append(name.get_text(strip=True))
    except:
        print('error scraping tbody')
player_name = [unicodedata.normalize("NFKD", x) for x in player_name]    
counter = 0
for row in table.find_all('tr'):
    # Find data for each column
    columns = row.find_all('td')      
    # print(columns)        
    if columns != [] and counter <= 14:
        player = player_name[counter]        
        counter += 1        
        # print('counter', counter)        
        number = columns[0].text.strip()        
        position = columns[1].text.strip()
        age = columns[2].text.split('-')[0]
        min_played = columns[3].text.strip()
        goals = int(columns[4].text.strip())
        assists = int(columns[5].text.strip())
        pk = int(columns[6].text.strip())
        pk_att = int(columns[7].text.strip())
        shoots = int(columns[8].text.strip())
        sot = int(columns[9].text.strip())
        yellow = int(columns[10].text.strip())
        red = int(columns[11].text.strip())
        touches = int(columns[12].text.strip())
        tackles = int(columns[13].text.strip())
        interceptions = int(columns[14].text.strip())
        blocks = columns[15].text.strip()                 
        passes_attempted = columns[22].text.strip()
        passes_completed = columns[21].text.strip()
        passes_completed_ratio = (columns[23].text.strip())        
        dribbles_attempted = int(columns[26].text.strip())
        dribbles_completed = int(columns[25].text.strip())
        # print('goals')
        # print(player, number, position)
        try:
            team_1 = team_1.append(
                {'Player': player,
                '#': number,
                'Pos': position,
                'Age': age,
                'Min': min_played,
                'Gls': goals,                
                'Ast': assists,
                'PK': pk,
                'PKatt': pk_att,
                'Sh': shoots,
                'SoT': sot,
                'CrdY': yellow,
                'CrdR': red,
                'Touches': touches,
                'Tkl': tackles,
                'Int': interceptions,
                'Blocks': blocks,
                'P_Att': passes_attempted,
                'P_Cmp': passes_completed,                
                'P_Cmp%': passes_completed_ratio,                
                'D_Att': dribbles_attempted,
                'D_Succ': dribbles_completed                
                }
                , ignore_index=True            
            )
            # print(f'Player {player} adicionado')
        except:
            print(f'error adding {player}')
team_1

,Player,#,Pos,Age,Min,Gls,Ast,PK,PKatt,Sh,...,CrdR,Touches,Tkl,Int,Blocks,P_Att,P_Cmp,P_Cmp%,D_Att,D_Succ
0,Marco Asensio,10,"FW,RW",26,65,0,0,0,0,1,...,0,39,0,0,0,34,27,79.4,1,0
1,Koke,8,RM,30,25,0,0,0,0,0,...,0,16,1,0,0,12,8,66.7,0,0
2,Dani Olmo,21,LW,24,90,0,0,0,0,2,...,0,61,0,1,3,44,34,77.3,4,0
3,Ferrán Torres,11,RW,22,53,0,0,0,0,2,...,0,26,0,0,0,20,11,55.0,2,0
4,Álvaro Morata,7,FW,30,37,1,0,0,0,1,...,0,13,0,0,1,7,3,42.9,1,0
5,Pedri,26,LM,20,90,0,0,0,0,0,...,0,74,1,2,2,63,56,88.9,7,4
6,Sergio Busquets,5,CM,34,90,0,0,0,0,0,...,0,50,2,0,1,43,40,93.0,1,1
7,Gavi,9,RM,18,65,0,0,0,0,0,...,0,20,4,0,1,12,9,75.0,0,0
8,Nico Williams,12,RW,20,25,0,0,0,0,0,...,0,17,1,0,1,9,7,77.8,4,1
9,Jordi Alba,18,LB,33,81,0,1,0,0,1,...,0,93,4,1,2,81,69,85.2,1,0


In [10]:
team_1_passes = team_1.sort_values(by='P_Cmp%', ascending=False).reset_index(drop="True")
team_1_passes.index = np.arange(1, len(team_1) +1)
team_1_passes[['Player', 'P_Att','P_Cmp%']]

,Player,P_Att,P_Cmp%
1,Sergio Busquets,43,93.0
2,Aymeric Laporte,103,91.3
3,Unai Simón,53,90.6
4,Pedri,63,88.9
5,Alejandro Balde,9,88.9
6,Dani Carvajal,74,87.8
7,Jordi Alba,81,85.2
8,Rodri,106,84.9
9,Marco Asensio,34,79.4
10,Nico Williams,9,77.8


In [11]:
table2 = soup.find('table', id='stats_c1e40422_summary')

In [12]:
team_2 = df[['Player', '#', 'Pos', 'Age', 'Min',
          'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks',
          'P_Att','P_Cmp', 'P_Cmp%',
          'D_Att', 'D_Succ'
            ]]
team_2

,Player,#,Pos,Age,Min,Gls,Ast,PK,PKatt,Sh,...,CrdR,Touches,Tkl,Int,Blocks,P_Att,P_Cmp,P_Cmp%,D_Att,D_Succ


In [13]:
player_name = []
for name in table2.tbody.find_all('th'):
    try:
        player_name.append(name.get_text(strip=True))
    except:
        print('error scraping tbody')
player_name = [unicodedata.normalize("NFKD", x) for x in player_name]
counter = 0
for row in table2.find_all('tr'):
    # Find data for each column
    columns = row.find_all('td')      
    # print(columns)        
    if columns != [] and counter <= 15:        
        player = player_name[counter]        
        # print(player)
        counter += 1        
        # print('counter', counter)
        number = columns[0].text.strip()        
        position = columns[1].text.strip()
        age = columns[2].text.split('-')[0]
        min_played = columns[3].text.strip()
        goals = int(columns[4].text.strip())
        assists = int(columns[5].text.strip())
        pk = int(columns[6].text.strip())
        pk_att = int(columns[7].text.strip())
        shoots = int(columns[8].text.strip())
        sot = int(columns[9].text.strip())
        yellow = int(columns[10].text.strip())
        red = int(columns[11].text.strip())
        touches = int(columns[12].text.strip())
        tackles = int(columns[13].text.strip())
        interceptions = int(columns[14].text.strip())
        blocks = columns[16].text.strip()                 
        passes_attempted = columns[22].text.strip()
        passes_completed = columns[21].text.strip()
        passes_completed_ratio = float(columns[23].text.strip())
        blocks = int(columns[24].text.strip())        
        dribbles_attempted = int(columns[26].text.strip())
        dribbles_completed = int(columns[25].text.strip())
        # print('goals')
        # print(player, number, position)
        try:
            team_2 = team_2.append(
                {'Player': player,
                '#': number,
                'Pos': position,
                'Age': age,
                'Min': min_played,
                'Gls': goals,                
                'Ast': assists,
                'PK': pk,
                'PKatt': pk_att,
                'Sh': shoots,
                'SoT': sot,
                'CrdY': yellow,
                'CrdR': red,
                'Touches': touches,
                'Tkl': tackles,
                'Int': interceptions,
                'Blocks': blocks,
                'P_Att': passes_attempted,
                'P_Cmp': passes_completed,                
                'P_Cmp%': passes_completed_ratio,                
                'D_Att': dribbles_attempted,
                'D_Succ': dribbles_completed                
                }
                , ignore_index=True            
            )
            # print(f'Player {player} adicionado')
        except:
            print(f'error adding {player}')
team_2

,Player,#,Pos,Age,Min,Gls,Ast,PK,PKatt,Sh,...,CrdR,Touches,Tkl,Int,Blocks,P_Att,P_Cmp,P_Cmp%,D_Att,D_Succ
0,Thomas Müller,13,FW,33,69,0,0,0,0,0,...,0,17,0,0,1,12,8,66.7,0,0
1,Niclas Füllkrug,9,FW,29,21,1,0,0,0,3,...,0,12,0,0,1,7,5,71.4,1,1
2,Jamal Musiala,14,"LW,AM",19,90,0,1,0,0,1,...,0,37,0,1,0,19,17,89.5,10,5
3,Serge Gnabry,10,"RW,LW",27,84,0,0,0,0,2,...,0,30,2,1,1,19,12,63.2,1,0
4,Jonas Hofmann,18,LW,30,6,0,0,0,0,0,...,0,4,0,0,0,2,1,50.0,0,0
5,İlkay Gündoğan,21,AM,32,69,0,0,0,0,0,...,0,27,3,1,0,18,12,66.7,1,1
6,Leroy Sané,19,RW,26,21,0,0,0,0,0,...,0,11,0,0,2,10,5,50.0,3,2
7,Leon Goretzka,8,DM,27,90,0,0,0,0,1,...,0,38,5,1,1,27,19,70.4,0,0
8,Joshua Kimmich,6,DM,27,90,0,0,0,0,2,...,0,52,3,1,3,39,29,74.4,2,1
9,David Raum,3,LB,24,86,0,0,0,0,0,...,0,62,2,5,2,48,34,70.8,1,1


In [14]:
team_2

,Player,#,Pos,Age,Min,Gls,Ast,PK,PKatt,Sh,...,CrdR,Touches,Tkl,Int,Blocks,P_Att,P_Cmp,P_Cmp%,D_Att,D_Succ
0,Thomas Müller,13,FW,33,69,0,0,0,0,0,...,0,17,0,0,1,12,8,66.7,0,0
1,Niclas Füllkrug,9,FW,29,21,1,0,0,0,3,...,0,12,0,0,1,7,5,71.4,1,1
2,Jamal Musiala,14,"LW,AM",19,90,0,1,0,0,1,...,0,37,0,1,0,19,17,89.5,10,5
3,Serge Gnabry,10,"RW,LW",27,84,0,0,0,0,2,...,0,30,2,1,1,19,12,63.2,1,0
4,Jonas Hofmann,18,LW,30,6,0,0,0,0,0,...,0,4,0,0,0,2,1,50.0,0,0
5,İlkay Gündoğan,21,AM,32,69,0,0,0,0,0,...,0,27,3,1,0,18,12,66.7,1,1
6,Leroy Sané,19,RW,26,21,0,0,0,0,0,...,0,11,0,0,2,10,5,50.0,3,2
7,Leon Goretzka,8,DM,27,90,0,0,0,0,1,...,0,38,5,1,1,27,19,70.4,0,0
8,Joshua Kimmich,6,DM,27,90,0,0,0,0,2,...,0,52,3,1,3,39,29,74.4,2,1
9,David Raum,3,LB,24,86,0,0,0,0,0,...,0,62,2,5,2,48,34,70.8,1,1


In [15]:
team_stats = df[['Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks',
          'P_Att','P_Cmp', 'P_Cmp%',
          'D_Att', 'D_Succ'
            ]]
team_stats['Team'] = 0
team_stats['Avg_Age'] = 0
team_stats = team_stats[list(('Team', 'Avg_Age','Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks',
          'P_Att','P_Cmp', 'P_Cmp%','D_Att', 'D_Succ'))]
team_stats

,Team,Avg_Age,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Tkl,Int,Blocks,P_Att,P_Cmp,P_Cmp%,D_Att,D_Succ


In [16]:
# SETTING UP TEAM NAMES

team_name = ['Spain', 'Germany']
team_name = [unicodedata.normalize("NFKD", x) for x in team_name]

In [17]:
team = team_name[0]
age = round(sum([int(x) for x in team_1['Age']])/len(team_1['Age']), 1)
goals = team_1['Gls'].sum()
assists = team_1['Ast'].sum()
pk = team_1['PK'].sum()
pk_att = sum([(x) for x in team_1['PKatt']])
shoots = team_1['Sh'].sum()
sot = team_1['SoT'].sum()
yellow = team_1['CrdY'].sum()
red = team_1['CrdR'].sum()
touches = team_1['Touches'].sum()
tackles = team_1['Tkl'].sum()
interceptions = team_1['Int'].sum()
blocks = team_1['Blocks'].sum()
passes_attempted = int(sum([int(x) for x in team_1['P_Att']]))
passes_completed = sum([int(x) for x in team_1['P_Cmp']])
passes_completed_ratio = round(passes_completed / passes_attempted * 100, 2)
dribbles_attempted = team_1['D_Att'].sum()
dribbles_completed = team_1['D_Succ'].sum()
# print('goals')
# print(player, number, position)
try:
    team_stats = team_stats.append(
        {'Team': team,
        'Avg_Age': age,        
        'Gls': goals,                
        'Ast': assists,
        'PK': pk,
        'PKatt': pk_att,
        'Sh': shoots,
        'SoT': sot,
        'CrdY': yellow,
        'CrdR': red,
        'Touches': touches,
        'Tkl': tackles,
        'Int': interceptions,
        'Blocks': blocks,
        'P_Att': passes_attempted,
        'P_Cmp': passes_completed,                
        'P_Cmp%': passes_completed_ratio,                
        'D_Att': dribbles_attempted,
        'D_Succ': dribbles_completed                
        }
        , ignore_index=True            
    )
    # print(f'Player {player} adicionado')
except:
    print(f'error adding {player}')       
# team_stats['P_Cmp%']=team_stats['P_Cmp%'].apply(lambda x:round(x,2))
# team_stats['P_Att']=team_stats['P_Att'].apply(lambda x:round(x,0))
# team_stats

In [18]:
team = team_name[1]
age = round(sum([int(x) for x in team_2['Age']])/len(team_2['Age']), 1)
goals = team_2['Gls'].sum()
assists = team_2['Ast'].sum()
pk = team_2['PK'].sum()
pk_att = sum([(x) for x in team_2['PKatt']])
shoots = team_2['Sh'].sum()
sot = team_2['SoT'].sum()
yellow = team_2['CrdY'].sum()
red = team_2['CrdR'].sum()
touches = team_2['Touches'].sum()
tackles = team_2['Tkl'].sum()
interceptions = team_2['Int'].sum()
blocks = team_2['Blocks'].sum()
passes_attempted = int(sum([int(x) for x in team_2['P_Att']]))
passes_completed = sum([int(x) for x in team_2['P_Cmp']])
passes_completed_ratio = round(passes_completed / passes_attempted * 100, 2)
dribbles_attempted = team_2['D_Att'].sum()
dribbles_completed = team_2['D_Succ'].sum()
# print('goals')
# print(player, number, position)
try:
    team_stats = team_stats.append(
        {'Team': team,
        'Avg_Age': age,        
        'Gls': goals,                
        'Ast': assists,
        'PK': pk,
        'PKatt': pk_att,
        'Sh': shoots,
        'SoT': sot,
        'CrdY': yellow,
        'CrdR': red,
        'Touches': touches,
        'Tkl': tackles,
        'Int': interceptions,
        'Blocks': blocks,
        'P_Att': passes_attempted,
        'P_Cmp': passes_completed,                
        'P_Cmp%': passes_completed_ratio,                
        'D_Att': dribbles_attempted,
        'D_Succ': dribbles_completed                
        }
        , ignore_index=True            
    )
    # print(f'Player {player} adicionado')
except:
    print(f'error adding {player}')

### Spain

In [19]:
team_1

,Player,#,Pos,Age,Min,Gls,Ast,PK,PKatt,Sh,...,CrdR,Touches,Tkl,Int,Blocks,P_Att,P_Cmp,P_Cmp%,D_Att,D_Succ
0,Marco Asensio,10,"FW,RW",26,65,0,0,0,0,1,...,0,39,0,0,0,34,27,79.4,1,0
1,Koke,8,RM,30,25,0,0,0,0,0,...,0,16,1,0,0,12,8,66.7,0,0
2,Dani Olmo,21,LW,24,90,0,0,0,0,2,...,0,61,0,1,3,44,34,77.3,4,0
3,Ferrán Torres,11,RW,22,53,0,0,0,0,2,...,0,26,0,0,0,20,11,55.0,2,0
4,Álvaro Morata,7,FW,30,37,1,0,0,0,1,...,0,13,0,0,1,7,3,42.9,1,0
5,Pedri,26,LM,20,90,0,0,0,0,0,...,0,74,1,2,2,63,56,88.9,7,4
6,Sergio Busquets,5,CM,34,90,0,0,0,0,0,...,0,50,2,0,1,43,40,93.0,1,1
7,Gavi,9,RM,18,65,0,0,0,0,0,...,0,20,4,0,1,12,9,75.0,0,0
8,Nico Williams,12,RW,20,25,0,0,0,0,0,...,0,17,1,0,1,9,7,77.8,4,1
9,Jordi Alba,18,LB,33,81,0,1,0,0,1,...,0,93,4,1,2,81,69,85.2,1,0


### Germany

In [20]:
team_2

,Player,#,Pos,Age,Min,Gls,Ast,PK,PKatt,Sh,...,CrdR,Touches,Tkl,Int,Blocks,P_Att,P_Cmp,P_Cmp%,D_Att,D_Succ
0,Thomas Müller,13,FW,33,69,0,0,0,0,0,...,0,17,0,0,1,12,8,66.7,0,0
1,Niclas Füllkrug,9,FW,29,21,1,0,0,0,3,...,0,12,0,0,1,7,5,71.4,1,1
2,Jamal Musiala,14,"LW,AM",19,90,0,1,0,0,1,...,0,37,0,1,0,19,17,89.5,10,5
3,Serge Gnabry,10,"RW,LW",27,84,0,0,0,0,2,...,0,30,2,1,1,19,12,63.2,1,0
4,Jonas Hofmann,18,LW,30,6,0,0,0,0,0,...,0,4,0,0,0,2,1,50.0,0,0
5,İlkay Gündoğan,21,AM,32,69,0,0,0,0,0,...,0,27,3,1,0,18,12,66.7,1,1
6,Leroy Sané,19,RW,26,21,0,0,0,0,0,...,0,11,0,0,2,10,5,50.0,3,2
7,Leon Goretzka,8,DM,27,90,0,0,0,0,1,...,0,38,5,1,1,27,19,70.4,0,0
8,Joshua Kimmich,6,DM,27,90,0,0,0,0,2,...,0,52,3,1,3,39,29,74.4,2,1
9,David Raum,3,LB,24,86,0,0,0,0,0,...,0,62,2,5,2,48,34,70.8,1,1


In [21]:
team_stats

,Team,Avg_Age,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Tkl,Int,Blocks,P_Att,P_Cmp,P_Cmp%,D_Att,D_Succ
0,Spain,25.7,1,1,0,0,7,3,1,0,774,17,8,003012111200200,670,569,84.93,21,6
1,Germany,27.5,1,1,0,0,11,4,3,0,502,29,13,17,381,287,75.33,19,11


In [22]:
round(sum([int(x) for x in team_2['Age']])/len(team_2['Age']), 1)

27.5

In [23]:
team_1_passes = team_1.sort_values(by='P_Cmp%', ascending=False)

In [24]:
team_1_passes

,Player,#,Pos,Age,Min,Gls,Ast,PK,PKatt,Sh,...,CrdR,Touches,Tkl,Int,Blocks,P_Att,P_Cmp,P_Cmp%,D_Att,D_Succ
6,Sergio Busquets,5,CM,34,90,0,0,0,0,0,...,0,50,2,0,1,43,40,93.0,1,1
11,Aymeric Laporte,24,CB,28,90,0,0,0,0,0,...,0,107,0,2,0,103,94,91.3,0,0
14,Unai Simón,23,GK,25,90,0,0,0,0,0,...,0,58,0,0,0,53,48,90.6,0,0
5,Pedri,26,LM,20,90,0,0,0,0,0,...,0,74,1,2,2,63,56,88.9,7,4
10,Alejandro Balde,14,LB,19,9,0,0,0,0,0,...,0,9,1,0,0,9,8,88.9,0,0
13,Dani Carvajal,20,RB,30,90,0,0,0,0,0,...,0,73,0,1,0,74,65,87.8,0,0
9,Jordi Alba,18,LB,33,81,0,1,0,0,1,...,0,93,4,1,2,81,69,85.2,1,0
12,Rodri,16,CB,26,90,0,0,0,0,0,...,0,118,3,1,2,106,90,84.9,0,0
0,Marco Asensio,10,"FW,RW",26,65,0,0,0,0,1,...,0,39,0,0,0,34,27,79.4,1,0
8,Nico Williams,12,RW,20,25,0,0,0,0,0,...,0,17,1,0,1,9,7,77.8,4,1


In [25]:
team_1.to_excel(r'C:\\Users\\marci\\Desktop\\WordCup2022\\Team_1.xlsx', index=False)

In [26]:
team_2.to_excel(r'C:\\Users\\marci\\Desktop\\WordCup2022\\Team_2.xlsx', index=False)

In [27]:
team_stats.to_excel(r'C:\\Users\\marci\\Desktop\\WordCup2022\\Team_stats.xlsx', index=False)

In [28]:
team_2[['Player', 'Sh', 'SoT']]

,Player,Sh,SoT
0,Thomas Müller,0,0
1,Niclas Füllkrug,3,1
2,Jamal Musiala,1,1
3,Serge Gnabry,2,0
4,Jonas Hofmann,0,0
5,İlkay Gündoğan,0,0
6,Leroy Sané,0,0
7,Leon Goretzka,1,0
8,Joshua Kimmich,2,1
9,David Raum,0,0
